<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#CAGED" data-toc-modified-id="CAGED-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>CAGED</a></span><ul class="toc-item"><li><span><a href="#Introdução/Introduction" data-toc-modified-id="Introdução/Introduction-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introdução/Introduction</a></span></li><li><span><a href="#Download" data-toc-modified-id="Download-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Download</a></span></li></ul></li><li><span><a href="#Tratamento-dos-dados" data-toc-modified-id="Tratamento-dos-dados-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Tratamento dos dados</a></span><ul class="toc-item"><li><span><a href="#Bibliotecas-utilizadas" data-toc-modified-id="Bibliotecas-utilizadas-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Bibliotecas utilizadas</a></span></li><li><span><a href="#Download-dos-Arquivos" data-toc-modified-id="Download-dos-Arquivos-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Download dos Arquivos</a></span></li><li><span><a href="#Tratamento-dos-Dados" data-toc-modified-id="Tratamento-dos-Dados-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Tratamento dos Dados</a></span><ul class="toc-item"><li><span><a href="#Preparação-do-Layout" data-toc-modified-id="Preparação-do-Layout-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Preparação do Layout</a></span></li><li><span><a href="#Leitura-dos-arquivos" data-toc-modified-id="Leitura-dos-arquivos-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>Leitura dos arquivos</a></span></li><li><span><a href="#Leitura-dos-Arquivos-CAGEDEST" data-toc-modified-id="Leitura-dos-Arquivos-CAGEDEST-2.3.3"><span class="toc-item-num">2.3.3&nbsp;&nbsp;</span>Leitura dos Arquivos CAGEDEST</a></span></li><li><span><a href="#Aplicação-dos-deparas-na-tabela-do-CAGEDEST" data-toc-modified-id="Aplicação-dos-deparas-na-tabela-do-CAGEDEST-2.3.4"><span class="toc-item-num">2.3.4&nbsp;&nbsp;</span>Aplicação dos deparas na tabela do CAGEDEST</a></span></li></ul></li><li><span><a href="#Limpeza-final-da-tabela" data-toc-modified-id="Limpeza-final-da-tabela-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Limpeza final da tabela</a></span></li></ul></li><li><span><a href="#Exportação" data-toc-modified-id="Exportação-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Exportação</a></span></li><li><span><a href="#Versão-Resumida" data-toc-modified-id="Versão-Resumida-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Versão Resumida</a></span></li></ul></div>

# CAGED
## Introdução/Introduction
Os dados que serão tratados abaixo são dados do CAGED (Cadastro Geral de Empregados e Desempregados) http://trabalho.gov.br/trabalhador-caged e servirão de apoio a análises

## Download
Os dados estão disponíveis no link ftp://ftp.mtps.gov.br/pdet/microdados/CAGED e possui dados desde 2007
Esses dados são atualizados mensalmente e disponibilizados no formato *.7z* com tamanho aproximado de 24MB. Existe também um arquivo chamado *CAGEDEST_layout_Atualizado.xls* de 1MB que nos auxiliará no tratamento dos arquivos

# Tratamento dos dados
##  Bibliotecas utilizadas

In [ ]:
import pandas as pd
import urllib
import os
import pyunpack
from glob import glob
from tqdm import tqdm
from bs4 import BeautifulSoup

## Download dos Arquivos

In [ ]:
# Baixa todos os arquivos
months = ['01','02','03','04','05','06','07','08','09','10','11','12']
years = ['2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']

for i,y in tqdm(enumerate(years)):
    for j,m in enumerate(months):
        file = 'CAGEDEST_'+m+y+'.7z'
        urllib.request.urlretrieve('ftp://ftp.mtps.gov.br/pdet/microdados/CAGED/{}/{}'.format(y,file),'CAGED/{}'.format(file))
        pyunpack.Archive('CAGED/{}'.format(file)).extractall('CAGED/')
        

## Tratamento dos Dados
### Preparação do Layout
Das planilhas de leiaute, temos:
- 0 -> Planilha de leiaute dos dados
- 1 -> Planilha de depara municípios/Estado
- 2-6 -> Planilhas de depara de CNAEs e Classes
- 7 -> Planilha de Agrupamento por Regiões (Complexo)
- 8-10 -> Planilha de depara de Bairros
- 11-12 -> Parecem dispensáveis

### Leitura dos arquivos
Foi criado um ```pd.ExcelFile``` a fim de facilitar depois as chamadas

In [ ]:
columns = [['CBO94','Ocupação'],
           ['CBO 2002 Ocupação','Ocupação'],
           ['CNAE 2.0 Subclas','Subclasse'],
           ['CNAE 2.0 Classe','Classe2'],
           ['CNAE 1.0 Classe','Classe1']]

In [ ]:
xls = pd.ExcelFile('CAGED/CAGEDEST_layout_Atualizado.xls')
sheets = xls.sheet_names

In [ ]:
# 0 - Planilha depara com informações gerais
leiaute_geral = pd.read_excel(xls, sheet_name=sheets[0], skiprows=1)
leiaute_geral.fillna(method='ffill', inplace=True)

# Selecionando as colunas com variáveis categoricas do CAGEDEST
col_num = [0,8,9,11,13,14,15,18,20,21,22,23,40,41]
col_list = []

for i in col_num:
    col_list.append(leiaute_geral['Nome'].unique()[i])
    
leiaute_geral.loc[leiaute_geral['Valor na Fonte']=='{', 'Valor na Fonte'] = '-1'

leiaute_geral['Valor na Fonte'] = leiaute_geral['Valor na Fonte'].replace('{','-1')

In [ ]:
# 1- Planilha depara municipios e estados
leiaute_mun = pd.read_excel(xls, sheet_name='municipio')
leiaute_mun = leiaute_mun['Município'].str.split(':',expand=True).rename(columns={0: "cod_municipio", 1: "dummy"})
leiaute_mun = pd.concat([leiaute_mun,
                         leiaute_mun['dummy'].str.split('-',1,expand=True)],
                         axis=1).rename(columns={0: "Estado", 1: "Municipio"})
leiaute_mun.drop(columns=['dummy'], inplace=True)
leiaute_mun['Estado'] = leiaute_mun['Estado'].str.upper()
leiaute_mun['cod_municipio'] = leiaute_mun['cod_municipio'].astype('int64')

In [ ]:
# 2 a 6 - Planilhas depara de CNAEs e Classes

# Colocar em um loop algum dia
leiaute_CBO = pd.read_excel(xls,sheet_name=sheets[2])
leiaute_CBO2 = pd.read_excel(xls,sheet_name=sheets[3])
leiaute_subcl = pd.read_excel(xls,sheet_name=sheets[4])
leiaute_classe2 = pd.read_excel(xls,sheet_name=sheets[5])
leiaute_classe1 = pd.read_excel(xls,sheet_name=sheets[6])

leiautes_2_6 = [leiaute_CBO, leiaute_CBO2, leiaute_subcl, leiaute_classe2, leiaute_classe1]

def tratamento_leiaute_1(leiaute1, index):
    
    leiaute = leiaute1.iloc[:,0].str.split(':',1,expand=True).rename(columns={0:columns[index][0], 1: columns[index][1]})

    return leiaute

for i,items in enumerate(leiautes_2_6):
    leiautes_2_6[i] = tratamento_leiaute_1(items,i)

In [ ]:
# 2 a 6 - Planilhas depara de CNAEs e Classes

# Colocar em um loop algum dia
leiaute_CBO = pd.read_excel(xls,sheet_name=sheets[2])
leiaute_CBO = leiaute_CBO.iloc[:,0].str.split(':',1,expand=True).rename(columns={0:columns[0][0], 1: columns[0][1]})
leiaute_CBO.drop(columns=[leiaute_CBO.columns[0]],inplace=True)

leiaute_CBO2 = pd.read_excel(xls,sheet_name=sheets[3])
leiaute_CBO2 = leiaute_CBO2.iloc[:,0].str.split(':',1,expand=True).rename(columns={0:columns[1][0], 1: columns[1][1]})
leiaute_CBO2['CBO 2002 Ocupação'] = leiaute_CBO2['CBO 2002 Ocupação'].astype('int64')

leiaute_subcl = pd.read_excel(xls,sheet_name=sheets[4])
leiaute_subcl = leiaute_subcl.iloc[:,0].str.split(':',1,expand=True).rename(columns={0:columns[2][0], 1: columns[2][1]})
leiaute_subcl['CNAE 2.0 Subclas'] = leiaute_subcl['CNAE 2.0 Subclas'].astype('int64')

leiaute_classe2 = pd.read_excel(xls,sheet_name=sheets[5])
leiaute_classe2 = leiaute_classe2.iloc[:,0].str.split(':',1,expand=True).rename(columns={0:columns[3][0], 1: columns[3][1]})
leiaute_classe2['CNAE 2.0 Classe'] = leiaute_classe2['CNAE 2.0 Classe'].astype('int64')

leiaute_classe1 = pd.read_excel(xls,sheet_name=sheets[6])
leiaute_classe1 = leiaute_classe1.iloc[:,0].str.split(':',1,expand=True).rename(columns={0:columns[4][0], 1: columns[4][1]})
leiaute_classe1['CNAE 1.0 Classe'] = leiaute_classe1['CNAE 1.0 Classe'].astype('int64')

In [ ]:
colunas =   {'cod_mesorregiao':"Mesorregião", 'cod_microrregiao':'Microrregião', 'cod_reg_adm_rj':'Região Adm RJ',
             'cod_reg_adm_sp':"Região Adm SP", 'cod_reg_gov_sp':'Região Gov SP', 'cod_reg_senai_sp':'Região Senai SP',
             'cod_reg_senac_pr':"Região Senac PR", 'cod_reg_senai_pr':'Região Senai PR', 
             'cod_sub_reg_senai_pr':'Região Senai PR','cod_corede_04':"Corede_04", 'cod_corede':'Corede'}

leiautes_7 = [['cod_mesorregiao','Mesorregião'],
              ['cod_microrregiao','Microrregião'],
              ['cod_reg_adm_rj','Região Adm RJ'],
              ['cod_reg_adm_sp','Região Adm SP'],
              ['cod_reg_gov_sp','Região Gov SP'],
              ['cod_reg_senai_sp','Região Senai SP'],
              ['cod_reg_senac_pr',"Região Senac PR"],
              ['cod_reg_senai_pr','Região Senai PR'],
              ['cod_sub_reg_senai_pr','Sub-Região Senai PR'], 
              ['cod_corede_04',"Região Corede 04"],
              ['cod_corede','Região Corede']]

In [ ]:
# 7 - Planilhas com Regiões
# Colocar em um loop algum dia
leiaute_reg = pd.read_excel(xls, sheet_name = sheets[7])
leiaute_reg.drop(columns=leiaute_reg.iloc[:,leiaute_reg.columns.str.contains('Unnamed*')], inplace=True)

leiaute_meso = pd.DataFrame(leiaute_reg.iloc[:,0].copy().dropna())
leiaute_meso = leiaute_meso.iloc[:,0].str.split(':',1,expand=True).rename(columns={0: leiautes_7[0][0],1:leiautes_7[0][1]})
leiaute_meso['cod_mesorregiao']=leiaute_meso['cod_mesorregiao'].astype('int')

leiaute_micro = pd.DataFrame(leiaute_reg.iloc[:,1].copy().dropna())
leiaute_micro = leiaute_micro.iloc[:,0].str.split(':',1,expand=True).rename(columns={0: leiautes_7[1][0],1:leiautes_7[1][1]})
leiaute_micro['cod_microrregiao']=leiaute_micro['cod_microrregiao'].astype('int')

leiaute_reg_adm_rj = pd.DataFrame(leiaute_reg.iloc[:,2].copy().dropna())
leiaute_reg_adm_rj = leiaute_reg_adm_rj.iloc[:,0].str.split(':',1,expand=True).rename(columns={0: leiautes_7[2][0],1:leiautes_7[2][1]})
leiaute_reg_adm_rj['cod_reg_adm_rj']=leiaute_reg_adm_rj['cod_reg_adm_rj'].astype('int')

leiaute_reg_adm_sp = pd.DataFrame(leiaute_reg.iloc[:,3].copy().dropna())
leiaute_reg_adm_sp = leiaute_reg_adm_sp.iloc[:,0].str.split(':',1,expand=True).rename(columns={0: leiautes_7[3][0],1:leiautes_7[3][1]})
leiaute_reg_adm_sp['cod_reg_adm_sp']=leiaute_reg_adm_sp['cod_reg_adm_sp'].astype('int')

leiaute_reg_gov_sp = pd.DataFrame(leiaute_reg.iloc[:,4].copy().dropna())
leiaute_reg_gov_sp = leiaute_reg_gov_sp.iloc[:,0].str.split(':',1,expand=True).rename(columns={0: leiautes_7[4][0],1:leiautes_7[4][1]})
leiaute_reg_gov_sp['cod_reg_gov_sp']=leiaute_reg_gov_sp['cod_reg_gov_sp'].astype('int')

leiaute_reg_senai_sp = pd.DataFrame(leiaute_reg.iloc[:,5].copy().dropna())
leiaute_reg_senai_sp = leiaute_reg_senai_sp.iloc[:,0].str.split(':',1,expand=True).rename(columns={0: leiautes_7[5][0],1:leiautes_7[5][1]})
leiaute_reg_senai_sp['cod_reg_senai_sp']=leiaute_reg_senai_sp['cod_reg_senai_sp'].astype('int')

leiaute_reg_senac_pr = pd.DataFrame(leiaute_reg.iloc[:,6].copy().dropna())
leiaute_reg_senac_pr = leiaute_reg_senac_pr.iloc[:,0].str.split(':',1,expand=True).rename(columns={0: leiautes_7[6][0],1:leiautes_7[6][1]})
leiaute_reg_senac_pr['cod_reg_senac_pr']=leiaute_reg_senac_pr['cod_reg_senac_pr'].astype('int')

leiaute_reg_senai_pr = pd.DataFrame(leiaute_reg.iloc[:,7].copy().dropna())
leiaute_reg_senai_pr = leiaute_reg_senai_pr.iloc[:,0].str.split(':',1,expand=True).rename(columns={0: leiautes_7[7][0],1:leiautes_7[7][1]})
leiaute_reg_senai_pr['cod_reg_senai_pr']=leiaute_reg_senai_pr['cod_reg_senai_pr'].astype('int')

leiaute_sub_reg_senai_pr = pd.DataFrame(leiaute_reg.iloc[:,8].copy().dropna())
leiaute_sub_reg_senai_pr = leiaute_sub_reg_senai_pr.iloc[:,0].str.split(':',1,expand=True).rename(columns={0: leiautes_7[8][0],1:leiautes_7[8][1]})
leiaute_sub_reg_senai_pr['cod_sub_reg_senai_pr']=leiaute_sub_reg_senai_pr['cod_sub_reg_senai_pr'].astype('int')

leiaute_corede_04 = pd.DataFrame(leiaute_reg.iloc[:,9].copy().dropna())
leiaute_corede_04 = leiaute_corede_04.iloc[:,0].str.split(':',1,expand=True).rename(columns={0: leiautes_7[9][0],1:leiautes_7[9][1]})
leiaute_corede_04['cod_corede_04']=leiaute_corede_04['cod_corede_04'].astype('int')

leiaute_corede = pd.DataFrame(leiaute_reg.iloc[:,10].copy().dropna())
leiaute_corede = leiaute_corede.iloc[:,0].str.split(':',1,expand=True).rename(columns={0: leiautes_7[10][0],1:leiautes_7[10][1]})
leiaute_corede['cod_corede']=leiaute_corede['cod_corede'].astype('int')

In [ ]:
# 8 a 10 - Planilhas de depara de Bairros SP, Fortaleza e RJ
leiaute_sp = pd.read_excel(xls, sheet_name=sheets[8], skiprows=1)
leiaute_sp['Valor na Fonte'] = leiaute_sp['Valor na Fonte'].astype('int')

leiaute_fort = pd.read_excel(xls, sheet_name=sheets[9], skiprows=1)
leiaute_fort['Valor na Fonte'] = leiaute_fort['Valor na Fonte'].astype('int')

leiaute_rj = pd.read_excel(xls, sheet_name=sheets[10], skiprows=1)
leiaute_rj['Valor na Fonte'] = leiaute_rj['Valor na Fonte'].astype('int')

In [ ]:
# 11 - Planilhas de depara de Distritos SP e DF
leiaute_sp_d = pd.read_excel(xls, sheet_name=sheets[11], names=('cod_dist','Distrito'), header=None)
leiaute_sp_d['cod_dist'] = leiaute_sp_d['cod_dist'].apply(lambda x: x.replace('{ñ class}','0')).astype('int')

leiaute_df = pd.read_excel(xls, sheet_name=sheets[12], names=('cod_dist','Distrito'))
leiaute_df['cod_dist'] = leiaute_df['cod_dist'].apply(lambda x: x.replace('{ñ class}','0')).astype('int')
leiaute_df['Distrito'] = leiaute_df['Distrito'].apply(lambda x: x.replace('{ñ class}','NAO SE APLICA'))

### Leitura dos Arquivos CAGEDEST

In [ ]:
# Leitura dos Arquivos
df = pd.read_csv('CAGED/CAGEDEST_012007.txt', encoding='Latin-1', sep=';',decimal=',', low_memory=False)

In [ ]:
convert_dict = {'Salário Mensal':'float', 'Tempo Emprego':'float', 'Bairros SP':'int','Bairros Fortaleza':'int',
               'Bairros RJ':'int','Distritos SP':'int','Região Adm RJ':'int','Região Adm SP':'int','Região Corede':'int',
               'Região Corede 04':'int','Região Gov SP':'int','Região Senac PR':'int','Região Senai PR':'int',
                'Região Senai SP':'int','Sub-Região Senai PR':'int','CBO 2002 Ocupação':'int','CNAE 1.0 Classe':'int',
                'CNAE 2.0 Classe':'int', 'CNAE 2.0 Subclas':'int', 'Distritos SP':'int','Regiões Adm DF':'int'}

df = df.replace(['^{ñ.*','^\*+','^\d+\-'],'0',regex=True).astype(convert_dict)

### Aplicação dos deparas na tabela do CAGEDEST

In [ ]:
# Junção das primeiras colunas
df = pd.merge(left=df, right=leiaute_mun, how='left', left_on=['Município'], right_on=['cod_municipio'], copy=False)
df = pd.merge(left=df, right=leiaute_CBO2, how='left',on=['CBO 2002 Ocupação'], copy=False)
df = pd.merge(left=df, right=leiaute_classe1, how='left',on=['CNAE 1.0 Classe'], copy=False)
df = pd.merge(left=df, right=leiaute_classe2, how='left',on=['CNAE 2.0 Classe'], copy=False)
df = pd.merge(left=df, right=leiaute_subcl, how='left',on=['CNAE 2.0 Subclas'], copy=False)

In [ ]:
# Junção das colunas de Bairro
df = pd.merge(left=df, right=leiaute_sp, how='left',
              left_on=['Bairros SP'], right_on=['Valor na Fonte'],
              suffixes=('','_sp'), copy=False)#.drop(columns=['Valor na Fonte'], inplace=True)

df = pd.merge(left=df, right=leiaute_fort, how='left',
              left_on=['Bairros Fortaleza'], right_on=['Valor na Fonte'],
              suffixes=('','_fort'), copy=False)#.drop(columns=['Valor na Fonte'], inplace=True)

df = pd.merge(left=df, right=leiaute_rj, how='left',
              left_on=['Bairros RJ'], right_on=['Valor na Fonte'],
              suffixes=('','_rj'), copy=False)#.drop(columns=['Valor na Fonte'], inplace=True)

In [ ]:
# Junção das colunas de Distrito
df = pd.merge(left=df, right=leiaute_sp_d, how='left',
              left_on=['Distritos SP'], right_on=['cod_dist'],
              suffixes=('','_dist_sp'), copy=False)

df = pd.merge(left=df, right=leiaute_df, how='left',
              left_on=['Regiões Adm DF'], right_on=['cod_dist'],
              suffixes=('','_dist_df'), copy=False)

In [ ]:
sub = ['Admitidos/Desligados', 'Faixa Empr Início Jan','Grau Instrução',
       'IBGE Subsetor','Ind Aprendiz','Ind Portador Defic', 'Raça Cor','Sexo','Tipo Estab',
       'Tipo Defic','Tipo Mov Desagregado','UF','Ind Trab Parcial','Ind Trab Intermitente']

for i,col in enumerate(col_list):
    temp_df = leiaute_geral[leiaute_geral['Nome']==col][['Nome','Categorias','Valor na Fonte']]
    try:
        df[sub[i]] = df[sub[i]].astype('int') 
        temp_df['Valor na Fonte'] = temp_df['Valor na Fonte'].astype('int')
        df = pd.merge(left=df, right=temp_df,
                      left_on=[sub[i]], right_on=['Valor na Fonte'],
                      suffixes=('',col), copy=False)
        print('{} Ok'.format(sub[i]))
    except:
        print('{} não ok'.format(sub[i]))

In [ ]:
tabelas = [[leiaute_meso,'cod_mesorregiao','Mesorregião'],
           [leiaute_micro,'cod_microrregiao','Microrregião'],
           [leiaute_reg_adm_rj,'cod_reg_adm_rj','Região Adm RJ'],
           [leiaute_reg_adm_sp,'cod_reg_adm_sp','Região Adm SP'],
           [leiaute_reg_gov_sp,'cod_reg_gov_sp','Região Gov SP'],
           [leiaute_reg_senai_sp,'cod_reg_senai_sp','Região Senai SP'],
           [leiaute_reg_senac_pr,'cod_reg_senac_pr',"Região Senac PR"],
           [leiaute_reg_senai_pr,'cod_reg_senai_pr','Região Senai PR'],
           [leiaute_sub_reg_senai_pr,'cod_sub_reg_senai_pr','Sub-Região Senai PR'], 
           [leiaute_corede_04,'cod_corede_04',"Região Corede 04"],
           [leiaute_corede,'cod_corede','Região Corede']]

for i,item in enumerate(tabelas):
    df = pd.merge(left=df, right=item[0], how='left',
              left_on=[item[2]], right_on=[item[1]], copy=False)

## Limpeza final da tabela

In [ ]:
df['Competência Declarada'] = df['Competência Declarada'].astype('object')
df = df.select_dtypes('object').copy()

drop_cols = []
[drop_cols.append(col) for col in df.columns if 'cod_' in col]
[drop_cols.append(col) for col in df.columns if 'Nome' in col]

df.drop(columns=drop_cols, inplace=True)

In [ ]:
# Renomeação das colunas
subs = {'Classe1':'CNAE1.0', 'Classe2':'CNAE2.0',
        'Subclasse':'Subclasse_CNAE2.0', 'Categorias':'Bairro_SP', 'Descrição':'Bairro_SP_desc',
        'Categorias_fort':'Bairro_Fortaleza', 'Descrição_fot':'Bairro_Fortaleza_desc',
        'Categorias_rj':'Bairro_RJ', 'Descrição_rj':'Bairro_RJ_desc', 'Distrito':'Distrito_SP',
        'Distrito_dist_df':'Distrito_DF', 'Mesorregião_y':'Mesorregião', 'Microrregião_y':'Microrregião'}

df.rename(subs, axis=1, inplace=True)
df.rename(columns=lambda x: x.replace('Categorias',''), inplace=True)
df.rename(columns=lambda x: x.replace('_y',''), inplace=True)

# Exportação

In [ ]:
# Gerando arquivo CSV sem índices
df.to_csv('path/cagedest.csv',index=False)

# Versão Resumida

In [ ]:
lista = glob('CAGED/CAGEDEST_*.txt')

In [ ]:
def tratamento(filepath):
    
    df = pd.read_csv(filepath, encoding='Latin-1', sep=';',decimal=',', low_memory=False)
    # Tratamento inicial das colunas do arquivo
    convert_dict = {'Salário Mensal':'float', 'Tempo Emprego':'float', 'Bairros SP':'int','Bairros Fortaleza':'int',
                   'Bairros RJ':'int','Distritos SP':'int','Região Adm RJ':'int','Região Adm SP':'int','Região Corede':'int',
                   'Região Corede 04':'int','Região Gov SP':'int','Região Senac PR':'int','Região Senai PR':'int',
                    'Região Senai SP':'int','Sub-Região Senai PR':'int','CBO 2002 Ocupação':'int','CNAE 1.0 Classe':'int',
                    'CNAE 2.0 Classe':'int', 'CNAE 2.0 Subclas':'int', 'Distritos SP':'int','Regiões Adm DF':'int'}

    df = df.replace(['^{ñ.*','^\*+','^\d+\-'],'0',regex=True).astype(convert_dict)
    
    # Junção das primeiras colunas
    df = pd.merge(left=df, right=leiaute_mun, how='left', left_on=['Município'], right_on=['cod_municipio'], copy=False)
    df = pd.merge(left=df, right=leiaute_CBO2, how='left',on=['CBO 2002 Ocupação'], copy=False)
    df = pd.merge(left=df, right=leiaute_classe1, how='left',on=['CNAE 1.0 Classe'], copy=False)
    df = pd.merge(left=df, right=leiaute_classe2, how='left',on=['CNAE 2.0 Classe'], copy=False)
    df = pd.merge(left=df, right=leiaute_subcl, how='left',on=['CNAE 2.0 Subclas'], copy=False)

    # Junção das colunas de Bairro
    df = pd.merge(left=df, right=leiaute_sp, how='left',
                  left_on=['Bairros SP'], right_on=['Valor na Fonte'],
                  suffixes=('','_sp'), copy=False)#.drop(columns=['Valor na Fonte'], inplace=True)

    df = pd.merge(left=df, right=leiaute_fort, how='left',
                  left_on=['Bairros Fortaleza'], right_on=['Valor na Fonte'],
                  suffixes=('','_fort'), copy=False)#.drop(columns=['Valor na Fonte'], inplace=True)

    df = pd.merge(left=df, right=leiaute_rj, how='left',
                  left_on=['Bairros RJ'], right_on=['Valor na Fonte'],
                  suffixes=('','_rj'), copy=False)#.drop(columns=['Valor na Fonte'], inplace=True)

    # Junção das colunas de Distrito
    df = pd.merge(left=df, right=leiaute_sp_d, how='left',
                  left_on=['Distritos SP'], right_on=['cod_dist'],
                  suffixes=('','_dist_sp'), copy=False)

    df = pd.merge(left=df, right=leiaute_df, how='left',
                  left_on=['Regiões Adm DF'], right_on=['cod_dist'],
                  suffixes=('','_dist_df'), copy=False)

    sub = ['Admitidos/Desligados', 'Faixa Empr Início Jan','Grau Instrução',
           'IBGE Subsetor','Ind Aprendiz','Ind Portador Defic', 'Raça Cor','Sexo','Tipo Estab',
           'Tipo Defic','Tipo Mov Desagregado','UF','Ind Trab Parcial','Ind Trab Intermitente']

    for i,col in enumerate(col_list):
        temp_df = leiaute_geral[leiaute_geral['Nome']==col][['Nome','Categorias','Valor na Fonte']]
        try:
            df[sub[i]] = df[sub[i]].astype('int') 
            temp_df['Valor na Fonte'] = temp_df['Valor na Fonte'].astype('int')
            df = pd.merge(left=df, right=temp_df,
                          left_on=[sub[i]], right_on=['Valor na Fonte'],
                          suffixes=('',col), copy=False)
        except:
            print('{} não ok, arquivo {}'.format(sub[i], filepath))

    # Junção das colunas de região
    tabelas = [[leiaute_meso,'cod_mesorregiao','Mesorregião'],
               [leiaute_micro,'cod_microrregiao','Microrregião'],
               [leiaute_reg_adm_rj,'cod_reg_adm_rj','Região Adm RJ'],
               [leiaute_reg_adm_sp,'cod_reg_adm_sp','Região Adm SP'],
               [leiaute_reg_gov_sp,'cod_reg_gov_sp','Região Gov SP'],
               [leiaute_reg_senai_sp,'cod_reg_senai_sp','Região Senai SP'],
               [leiaute_reg_senac_pr,'cod_reg_senac_pr',"Região Senac PR"],
               [leiaute_reg_senai_pr,'cod_reg_senai_pr','Região Senai PR'],
               [leiaute_sub_reg_senai_pr,'cod_sub_reg_senai_pr','Sub-Região Senai PR'], 
               [leiaute_corede_04,'cod_corede_04',"Região Corede 04"],
               [leiaute_corede,'cod_corede','Região Corede']]

    for i,item in enumerate(tabelas):
        df = pd.merge(left=df, right=item[0], how='left',
                  left_on=[item[2]], right_on=[item[1]], copy=False)

    # Limpeza da tabela, eliminação de colunas que estão sobrando e colunas com códigos

    df['Competência Declarada'] = df['Competência Declarada'].astype('object')
    df = df.select_dtypes('object').copy()

    drop_cols = []
    [drop_cols.append(col) for col in df.columns if 'cod_' in col]
    [drop_cols.append(col) for col in df.columns if 'Nome' in col]

    df.drop(columns=drop_cols, inplace=True)

    # Renomeação das colunas
    subs = {'Classe1':'CNAE1.0', 'Classe2':'CNAE2.0',
            'Subclasse':'Subclasse_CNAE2.0', 'Categorias':'Bairro_SP', 'Descrição':'Bairro_SP_desc',
            'Categorias_fort':'Bairro_Fortaleza', 'Descrição_fot':'Bairro_Fortaleza_desc',
            'Categorias_rj':'Bairro_RJ', 'Descrição_rj':'Bairro_RJ_desc', 'Distrito':'Distrito_SP',
            'Distrito_dist_df':'Distrito_DF', 'Mesorregião_y':'Mesorregião', 'Microrregião_y':'Microrregião'}

    df.rename(subs, axis=1, inplace=True)
    df.rename(columns=lambda x: x.replace('Categorias',''), inplace=True)
    df.rename(columns=lambda x: x.replace('_y',''), inplace=True)
    
    return df

In [ ]:
tabelao = pd.DataFrame()

for filepath in tqdm(lista):
    tab = tratamento(filepath)
    tab.to_csv(filepath.split('.')[0]+'.csv',index=False)
    tabelao.append(tab)